In [1]:
# Imports Cell
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.
/home/kyriakos/anaconda3/envs/char-sec2sec-translate/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kyriakos/anaconda3/envs/char-sec2sec-translate/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kyriakos/anaconda3/envs/char-sec2sec-translate/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np

In [2]:
# Basic Parameters
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = './ell.txt'

In [3]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 69
Number of unique output tokens: 110
Max sequence length for inputs: 25
Max sequence length for outputs: 49


In [5]:
# Initialize Model Arrays
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [6]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [7]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [8]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [9]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 17s 2ms/step - loss: 1.6371 - acc: 0.6298 - val_loss: 1.8454 - val_acc: 0.5035
Epoch 2/100
8000/8000 [==============================] - 17s 2ms/step - loss: 1.2328 - acc: 0.6811 - val_loss: 1.5008 - val_acc: 0.5957
Epoch 3/100
8000/8000 [==============================] - 19s 2ms/step - loss: 0.9813 - acc: 0.7331 - val_loss: 1.2788 - val_acc: 0.6476
Epoch 4/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.8555 - acc: 0.7598 - val_loss: 1.1662 - val_acc: 0.6697
Epoch 5/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.7803 - acc: 0.7780 - val_loss: 1.0979 - val_acc: 0.6864
Epoch 6/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.7228 - acc: 0.7930 - val_loss: 1.0352 - val_acc: 0.7046
Epoch 7/100
8000/8000 [================

Epoch 53/100
8000/8000 [==============================] - 6s 812us/step - loss: 0.1318 - acc: 0.9598 - val_loss: 0.9883 - val_acc: 0.7870
Epoch 54/100
8000/8000 [==============================] - 7s 813us/step - loss: 0.1286 - acc: 0.9607 - val_loss: 0.9847 - val_acc: 0.7884
Epoch 55/100
8000/8000 [==============================] - 7s 818us/step - loss: 0.1261 - acc: 0.9613 - val_loss: 0.9903 - val_acc: 0.7890
Epoch 56/100
8000/8000 [==============================] - 7s 817us/step - loss: 0.1228 - acc: 0.9623 - val_loss: 1.0101 - val_acc: 0.7876
Epoch 57/100
8000/8000 [==============================] - 7s 827us/step - loss: 0.1199 - acc: 0.9631 - val_loss: 1.0167 - val_acc: 0.7875
Epoch 58/100
8000/8000 [==============================] - 7s 849us/step - loss: 0.1173 - acc: 0.9638 - val_loss: 1.0227 - val_acc: 0.7867
Epoch 59/100
8000/8000 [==============================] - 7s 865us/step - loss: 0.1150 - acc: 0.9644 - val_loss: 1.0231 - val_acc: 0.7880
Epoch 60/100
8000/8000 [==========

In [10]:
# Save model
model.save('s2s.h5')

/home/kyriakos/anaconda3/envs/char-sec2sec-translate/lib/python3.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [11]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states


In [12]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [13]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [14]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [15]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Πάμε.

-
Input sentence: Run!
Decoded sentence: Τρέξε!

-
Input sentence: Run!
Decoded sentence: Τρέξε!

-
Input sentence: Who?
Decoded sentence: Ποιος;

-
Input sentence: Wow!
Decoded sentence: Ουάου!

-
Input sentence: Help!
Decoded sentence: Βοήθεια!

-
Input sentence: Jump!
Decoded sentence: Πήδα!

-
Input sentence: Hello!
Decoded sentence: Γεια σας.

-
Input sentence: Hurry!
Decoded sentence: Βιάσου!

-
Input sentence: I try.
Decoded sentence: Προσπαθώ.

-
Input sentence: I won!
Decoded sentence: Κέρδισα!

-
Input sentence: I won!
Decoded sentence: Κέρδισα!

-
Input sentence: Smile.
Decoded sentence: Χαμογέλα.

-
Input sentence: Attack!
Decoded sentence: Επίθεση!

-
Input sentence: Cheers!
Decoded sentence: Στην υγειά μας!

-
Input sentence: I fell.
Decoded sentence: Έπεσα.

-
Input sentence: I know.
Decoded sentence: Ξέρω.

-
Input sentence: I left.
Decoded sentence: Έφυγα.

-
Input sentence: I lied.
Decoded sentence: Είπα ψέματα.

-
Input 